测试: 缺陷检测

In [7]:
# 步骤1：导入使用的库，读取原始图像并显示
import cv2
import numpy as np

img = cv2.imread(r'./dataset3/pill2.jpg')
cv2.imshow('original', img)

# 等待用户按键
cv2.waitKey(0)
cv2.destroyAllWindows()

# 步骤2：预处理
# 该步骤主要是为了方便进行后续处理，主要包含色彩空间转换处理、阈值处理、形态学开运算
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imshow('thresh', thresh)
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
opening1 = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)  # 形态学开运算
cv2.imshow('opening1', opening1)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 步骤3：使用距离变换函数distance Transform确定前景
# 该步骤使用距离变换函数确定每一个像素点距离最近背景点的距离
dist_transform = cv2.distanceTransform(opening1, cv2.DIST_L2, 3)
ret, fore = cv2.threshold(dist_transform, 0.3 * dist_transform.max(), 255, 0)
cv2.imshow('fore', fore)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 步骤4：去噪处理
# 对确定的前景点进行再次处理，去掉图像内的噪声信息，通过开运算完成。
kernel = np.ones((3, 3), np.uint8)
opening2 = cv2.morphologyEx(fore, cv2.MORPH_OPEN, kernel)  # 形态学开运算
cv2.imshow('opening2', opening2)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 步骤5：提取轮廓
# 使用轮廓提取函数提取上述处理结果图像内的轮廓
opening2 = np.array(opening2, np.uint8)
contours, hierarchy = cv2.findContours(opening2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# 步骤6：缺陷检测
# 计算面积：分别计算外接圆面积A和轮廓面积B
# 面积比较：计算B:A的比值，并根据比值进行判断。若比值大于阈值 T，则说明轮廓与外接圆较一致，当前对象是圆形的
# 否则，当前对象是残缺品，是次品
count = 0
font = cv2.FONT_HERSHEY_COMPLEX
for cnt in contours:
    (x, y), radius = cv2.minEnclosingCircle(cnt)
    center = (int(x), int(y))
    radius = int(radius)
    circle_img = cv2.circle(img, center, radius, (255, 255, 255), 1)
    area = cv2.contourArea(cnt)
    area_circle = 3.14 * radius * radius
    if area / area_circle >= 0.5:
        img = cv2.putText(img, 'OK', center, font, 1, (255, 255, 255), 3)
    else:
        img = cv2.putText(img, 'BAD', center, font, 1, (255, 255, 255), 3)
    count += 1

img = cv2.putText(img, 'sum=' + str(count), (20, 50), font, 1, (255, 255, 255), 3)

# 步骤7：显示最终结果图像
cv2.imshow('result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 在测试图像'pill2.jpg'的结果图像中，
# 有 Fixme 个目标标记OK，有 Fixme 个目标标记BAD。